## Les paramêtres de notre simulation:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Mv_air=1.225
g=9.81
accelerations_moyenne=[5.1,5,5.3,5.2,5.8,5]
masses=[1760,1615,1498,1385,1540,1600]
Usurf=0.1
Cx=[0.38,0.29,0.35,0.28,0.34,0.28]
Cz=0.3
Sx=[2.6325,2.2987,2.444,2.1525,2.4344,2.6788]
Sz=[10.296,8.1631,8.8736,7.525,8.234,8.1631]
NOS=False
Ailes=False
Rank=int(input("Entrez le numéro de la voiture: 1 pour la Dodge,2 pour la Toyota,3 pour la Chevrolet,4 pour la Mazda, 5 pour la Nissan, 6 pour la Mitsubichi", ))-1
Indice_accessoires=int(input("entrez 1 pour les ailes, 2 pour la nitro, 3 si les deux sont utilisés, et n'importe quoi d'autre si aucun des deux ne sont utilisés", ))
if(Indice_accessoires==1 or Indice_accessoires==2 or Indice_accessoires==3):
    if(Indice_accessoires==3):
        NOS=True
        Ailes=True
    elif(Indice_accessoires==2):
        NOS=True
    else:
        Ailes=True
        
        
#Définition d'un pas temporel
Pas_temporel=0.001

# Attribution de la nitro à une partie précise de la course
Partie_Nitro=0
if(NOS==True):
    Partie_Nitro=int(input("Sur qu'elle partie de la piste, la nitro est elle utilisée (1 pour la pente, 2 pour le looping, 3 pour la fin de piste)", ))

#Modification de la masse, du coefficient de portance , de trainé, lié aux ailes
if(Ailes==True):
    for i in range(0,len(masses)):
        masses[i]+=45
        Sz[i]+=0.8
        Cx[i]=Cx[i]*0.95
        
    Cz=Cz*1.1
#Fin du paramètrage   

# Pente

In [ ]:

# En utilisant la méthode d'euler et un pas petit, on est capable de fournir une analyse précise de notre vitesse en bas de pente
#Rappel: Vn+1=Vn+deltaV*pas

#On définit le multiplicateur boost lié à la nitro, si on l'a séléctionné pour cette partie boost augmente de 30% l'accélération moyenne de la voiture(a*1.3)
if(Partie_Nitro==1):
    boost=1.3
else:
    boost=1

angle=3.69*np.pi/180
distance_de_fin=31
vitesse=0
distance=0
indice_temps_pente=0


while(distance<distance_de_fin):#Tant que nous ne sommes pas au bout de la piste:
    #Calcul de la variation de vitesse : Vn+1=Vn+(gsin(a)-ugcos(a)-(p*Cx*surface*Vn²/2m)+ am)*Pas_temporel
    vitesse=vitesse+(accelerations_moyenne[Rank]*boost+g*np.sin(angle)-Usurf*g*np.cos(angle)-((Mv_air*Cx[Rank]*Sx[Rank]*vitesse*vitesse)/(2*masses[Rank])))*Pas_temporel
    #Calcul de la distance actuelle
    distance=distance+(vitesse*Pas_temporel)
    #Le compteur de temps
    indice_temps_pente+=Pas_temporel

vitesse_fin_pente=vitesse


print("La vitesse en fin de piste est:",vitesse_fin_pente,"m/s")#La vitesse finale
print("La fin de piste fut atteinte en",indice_temps_pente,"Secondes")#La durée nécessaire pour traverser la pente

# Looping

In [ ]:
Distance_initiale=0
vitesse=12.773#vitesse_fin_pente
R=6 #Rayon de notre looping
Distance_max=2*np.pi*R #Le périmêtre de notre looping
angle=0 #On crée une variable qui suivra notre angle au fur et à mesure de notre tour
indice_temps_looping=0

if(Partie_Nitro==2):
    boost=1.3
else:
    boost=1

#On crée la liste qui nous permet de stocker les valeurs de la vitesse
Liste_vitesse=[]
#On crée la liste qui stocke le temps(On ne fait pas cela directement car on ne sait pas combient de temps dure notre looping)
Liste_suivi_temporel=[]

while(Distance_initiale<Distance_max):
    angle=(Distance_initiale/Distance_max)*2*np.pi # En effet, comme on part du principe que notre looping est un cercle parfait, il est possible de faire un rapport entre la position actuelle et le périmêtre pour obtenir l'angle théta
    # Calcul de la vitesse en utilisant la méthode de Euler(Seul l'accélération tengentielle est mesuré):
    vitesse=vitesse+(-g*np.sin(angle)-Usurf*((vitesse*vitesse/R)+g*np.cos(angle))+accelerations_moyenne[Rank]*boost-(Mv_air*Cx[Rank]*Sx[Rank]*vitesse*vitesse)/(2*masses[Rank]))*Pas_temporel
    Liste_vitesse.append(vitesse)
    Distance_initiale=Distance_initiale+vitesse*Pas_temporel
    indice_temps_looping+=Pas_temporel
    Liste_suivi_temporel.append(indice_temps_looping)
    if(Distance_initiale/(Distance_max/2)<1.0001) and (Distance_initiale/(Distance_max/2)>0.9999):
        if(vitesse>= np.sqrt(g*R)):
            print("looping passé avec succès",indice_temps_looping)
        else:
            print("Crash",indice_temps_looping)
       
vitesse_fin_looping=vitesse

print("La vitesse en fin de looping est de",vitesse_fin_looping,"m/s")
print("Le looping a été complété en",indice_temps_looping,"Secondes")

#Calcul de la vitesse nécessaire à compléter le looping


#ON affiche l'évolution en fonction du temps
plt.plot(Liste_suivi_temporel,Liste_vitesse)
plt.xlabel("Temps en $s$")
plt.ylabel("Vitesse en $m/s$")
plt.title("Evolution de la vitesse en fonction du temps")
plt.show()

# Ravin

In [ ]:
# Attributions des paramêtres
vitesse_en_x=vitesse_fin_looping
vitesse_en_y=0
distance_initiale=0
distance_max=9
hauteur=1
indice_temps_ravin=0

# Créations des listes qui suivront la trajectoire du véhicule
Liste_distances=[]
Liste_hauteurs=[]


while(hauteur>0):# Tant que nous ne touchons pas le sol
    # Calcul de la vitesse et de la position sur l'axe x avec la méthode de Euler: Vn+1=Vn+(-(Mv_air*Sx*Cx*vitesse_en_x²)/(2*masses)*Pas_temporel
    vitesse_en_x=vitesse_en_x+(-(Mv_air*Sx[Rank]*Cz*vitesse_en_x*vitesse_en_x)/(2*masses[Rank]))*Pas_temporel
    distance_initiale=distance_initiale+vitesse_en_x*Pas_temporel
    Liste_distances.append(distance_initiale)
    # Calcul de la vitesse et de la position sur l'axe y avec la méthode de Euler: Vn+1=Vn+(-g-(Mv_air*Cz*Sz*vitesse_en_y²)/(2*masses)*Pas_temporel
    vitesse_en_y=vitesse_en_y+(-g-(Mv_air*Sz[Rank]*Cz*vitesse_en_y*vitesse_en_y)/(2*masses[Rank]))*Pas_temporel
    hauteur=hauteur+vitesse_en_y*Pas_temporel
    Liste_hauteurs.append(hauteur)
    indice_temps_ravin+=Pas_temporel

if(distance_initiale>9):
    print("Voiture arrive à passer le ravin, avec un dépassement de",distance_initiale-9)
else:
    print("C'est le crash à",9-distance_initiale,"Mètres de la fin de la partie")

    
print("La vitesse de la voiture à la fin du ravin est de",vitesse_en_x,"m/s")
print("Ce segment à pris",indice_temps_ravin,"Secondes")

# On fait une représentation graphique de la trajectoire
print("Tracé de la trajectoire de notre véhicule:")

plt.plot(Liste_distances,Liste_hauteurs)
plt.xlabel("Distance en mètres")
plt.ylabel("Hauteur en mètres")
plt.title("Représentation de la trajectoire")
plt.xlim(0,10)
plt.ylim(0,10)
plt.show()

# Fin de piste

In [ ]:
vitesse=vitesse_en_x #On reprends la vitesse à la fin du ravin
distance_initiale=distance_initiale-9 # On sait que si la voiture réussit le saut, elle ne retombera pas précisément après 9 mêtres.
indice_temps_fin=0
distance_finale_fin=10

if(Partie_Nitro==3):
    boost=1.3
else:
    boost=1

#Calcul de la distance et de la durée de la partie de circuit
while(distance_initiale < distance_finale_fin):#Tant que nous ne sommes pas au bout de la piste:
    #Calcul de la variation de vitesse : Vn+1=Vn+(accelerations_moyenne-Usurf*g-(Mv_air*Sx*Cx*vitesse²/2masses)*Pas_temporel
    vitesse=vitesse+(accelerations_moyenne[Rank]*boost-Usurf*g-(Mv_air*Sx[Rank]*Cx[Rank]*vitesse*vitesse/(2*masses[Rank])))*Pas_temporel
    #Calcul de la distance actuelle
    distance_initiale=distance_initiale+(vitesse*Pas_temporel)
    #Le compteur de temps
    indice_temps_fin+=Pas_temporel

print("La vitesse en fin de piste est:",vitesse,"m/s")#La vitesse finale
print("La fin de piste fut atteinte en",indice_temps_fin,"Secondes")#La durée nécessaire pour traverser la pente

#Durée totale du circuit
print("La durée totale du circuit avec le modèle",Rank+1,"Est de",indice_temps_pente+indice_temps_looping+indice_temps_ravin+indice_temps_fin,"Secondes")

# Fin de piste

In [ ]:
vitesse=vitesse_en_x #On reprends la vitesse à la fin du ravin
distance_initiale=distance_initiale-9 # On sait que si la voiture réussit le saut, elle ne retombera pas précisément après 9 mêtres.
indice_temps_fin=0
distance_finale_fin=10

if(Partie_Nitro==3):
    boost=1.3
else:
    boost=1

#Calcul de la distance et de la durée de la partie de circuit
while(distance_initiale < distance_finale_fin):#Tant que nous ne sommes pas au bout de la piste:
    #Calcul de la variation de vitesse : Vn+1=Vn+(accelerations_moyenne-Usurf*g-(Mv_air*Sx*Cx*vitesse²/2masses)*Pas_temporel
    vitesse=vitesse+(accelerations_moyenne[Rank]*boost-Usurf*g-(Mv_air*Sx[Rank]*Cx[Rank]*vitesse*vitesse/(2*masses[Rank])))*Pas_temporel
    #Calcul de la distance actuelle
    distance_initiale=distance_initiale+(vitesse*Pas_temporel)
    #Le compteur de temps
    indice_temps_fin+=Pas_temporel

print("La vitesse en fin de piste est:",vitesse,"m/s")#La vitesse finale
print("La fin de piste fut atteinte en",indice_temps_fin,"Secondes")#La durée nécessaire pour traverser la pente

#Durée totale du circuit
print("La durée totale du circuit avec le modèle",Rank+1,"Est de",indice_temps_pente+indice_temps_looping+indice_temps_ravin+indice_temps_fin,"Secondes")

# Vitesse minimale en début de looping

### Bien que des formules existent, il est mieux d'utiliser notre simulation, qui prends les frottements,
### comme on connait la formule qui nous donne la vitesse minimale en haut du looping, il faut faire la simulation dans le sens inverse

In [ ]:
R=6
distance_initiale=np.pi*R
distance_finale =0
distance_looping=2*np.pi*R
angle=np.pi #On commence en haut du looping
vitesse=-np.sqrt(g*R) #D'après la formule trouvé lors du livrable 2 la vitesse est stratégiquement définie comme négative
duration=0

Vit=[]
t=[]

while(distance_initiale>distance_finale):#On fait une accélération inverse à précédamment, il nous suffit simplement de changer de signe de l'addition vers la soustraction et le tour est joué, nous accédons au passé
    angle=(distance_initiale/distance_looping)*2*np.pi
    vitesse=vitesse+(-g*np.sin(angle)-Usurf*((vitesse*vitesse/R)+g*np.cos(angle))+accelerations_moyenne[Rank]*boost-(Mv_air*Cx[Rank]*Sx[Rank]*vitesse*vitesse)/(2*masses[Rank]))*Pas_temporel
    Vit.append(vitesse)
    distance_initiale=distance_initiale+vitesse*Pas_temporel
    duration+=Pas_temporel
    t.append(duration)
    
print("La vitesse minimale en d'entrée en looping semble être de",-vitesse+0.003,"m/s")# La simulation marche , mais avec 0.003m/s d'imprecision
print(duration,"Sec")

plt.plot(t,Vit)
plt.show()